In [31]:
import random
from reconchess import *
import json
import pprint
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import re
from preprocessing.RBCDataset import RBCDataset

In [3]:
game_history = GameHistory.from_file("./RandomBot-TroutBot-black-2025_03_18-06_50_14.json")

In [4]:
# Testing GameHistory methods

# Player names
print(f'White player name: {game_history.get_white_player_name()}')
print(f'Black Player name: {game_history.get_black_player_name()}')

first_move = list(game_history.turns())[0]
arbitrary_move = list(game_history.turns())[2]
next_move = list(game_history.turns())[3]
last_move = list(game_history.turns())[-1]

# Turns data
print(f'---------------------------------------------------------------------------------------------')
print(f'There were no turns this game: {game_history.is_empty()}')
print(f'There were {game_history.num_turns()} this game')
print(f'The list of turns this game are: {list(game_history.turns())}')
print(f'Is this the first turn? {game_history.is_first_turn(first_move)}')
print(f'First turn: {game_history.first_turn()}')
print(f'Is this the last turn? {game_history.is_last_turn(last_move)}')
print(f'Last turn: {game_history.last_turn()}')

# Winner utility
print(f'---------------------------------------------------------------------------------------------')
print(f'The winning color: {game_history.get_winner_color()}')
print(f'Win condition: {game_history.get_win_reason()}')

# Sensing Utility
print(f'---------------------------------------------------------------------------------------------')
print(f'Turn has sense: {game_history.has_sense(arbitrary_move)}')
print(f'Sense of a particular turn: {game_history.sense(arbitrary_move)}')
print(f'Sense result of a sensing action: {game_history.sense_result(arbitrary_move)}')

# Moving Utility
print(f'---------------------------------------------------------------------------------------------')
print(f'Turn has move: {game_history.has_move(arbitrary_move)}')
print(f'Requested move of turn: {game_history.requested_move(arbitrary_move)}')
print(f'Taken move of turn: {game_history.taken_move(arbitrary_move)}')
print(f'Square I captured an opponent piece: {game_history.capture_square(arbitrary_move)} [NOTE: May be none of no piece captured this turn]')
print(f'Summarize move phase of turn: {game_history.move_result(arbitrary_move)}')

# Ground Truth methods
print(f'---------------------------------------------------------------------------------------------')
print(f'Truth before move (Forsyth–Edwards Notation): {game_history.truth_fen_before_move(arbitrary_move)}')
print(f'Truth before move: {game_history.truth_board_before_move(arbitrary_move)}')
print(f'Truth after move: (Forsyth-Edwards Notatin): {game_history.truth_fen_after_move(arbitrary_move)}')
print(f'Truth after move: {game_history.truth_board_after_move(arbitrary_move)}')
print(f'Map function: {game_history.collect(game_history.sense, list(game_history.turns()))}')



White player name: RandomBot
Black Player name: TroutBot
---------------------------------------------------------------------------------------------
There were no turns this game: False
There were 24 this game
The list of turns this game are: [Turn(white, 0), Turn(black, 0), Turn(white, 1), Turn(black, 1), Turn(white, 2), Turn(black, 2), Turn(white, 3), Turn(black, 3), Turn(white, 4), Turn(black, 4), Turn(white, 5), Turn(black, 5), Turn(white, 6), Turn(black, 6), Turn(white, 7), Turn(black, 7), Turn(white, 8), Turn(black, 8), Turn(white, 9), Turn(black, 9), Turn(white, 10), Turn(black, 10), Turn(white, 11), Turn(black, 11)]
Is this the first turn? True
First turn: Turn(white, 0)
Is this the last turn? True
Last turn: Turn(black, 11)
---------------------------------------------------------------------------------------------
The winning color: False
Win condition: WinReason.KING_CAPTURE
---------------------------------------------------------------------------------------------
Turn

In [27]:
def replace_numbers_with_u(strings):
    def replacer(match):
        return 'u' * int(match.group())  # Convert number to integer and repeat 'u' that many times

    return [re.sub(r'\d+', replacer, s) for s in strings]

# Write function to convert FEN string to Tensor
def FEN2InputTensor(fen_string, my_color):
    #my_color = 'white'
    fen_pattern = re.compile(r"^([rnbqkpRNBQKP1-8/]+) ([wb]) ([KQkq-]+) ([a-h1-8-]+) (\d+) (\d+)$")
    if my_color == 'white':
        piece2encoding = {
            'P' : 1,
            'R' : 2,
            'N' : 3,
            'B' : 4,
            'Q' : 5,
            'K' : 6,
        }
    elif my_color == 'black':
        piece2encoding = {
            'p' : 1,
            'r' : 2,
            'n' : 3,
            'b' : 4,
            'q' : 5,
            'k' : 6,
        }

    
    match = fen_pattern.match(fen_string)

    
    print(f'Input string: {fen_string}')
    
    if match:
        piece_placement, active_color, castling, en_passant, halfmove, fullmove = match.groups()
        print("Piece Placement:", piece_placement)
        print("Active Color:", active_color)
        print("Castling Rights:", castling)
        print("En Passant Target:", en_passant)
        print("Halfmove Clock:", halfmove)
        print("Fullmove Number:", fullmove)

        pieces_by_row = piece_placement.split('/')

        pieces_by_row = replace_numbers_with_u(pieces_by_row)

        # 7 is the value for unknown spaces.
        input_tensor = torch.full((8, 8), 7)

        for (rank, row) in zip(reversed(range(input_tensor.shape[0])), pieces_by_row):  # rows
            for (file, piece) in zip(reversed(range(input_tensor.shape[1])), row):  # columns
                if piece in piece2encoding.keys():
                    input_tensor[rank][file] = piece2encoding[piece]

        print(f'Input Tensor: {input_tensor}')

        return input_tensor
        
    else:
        print("Invalid FEN string")


input_tensor = FEN2InputTensor(game_history.truth_fen_before_move(arbitrary_move), my_color = 'white')

Input string: rnbqkbnr/pppp1ppp/8/4p3/8/8/PPPPPPPP/RNBQKBNR w KQkq e6 0 2
Piece Placement: rnbqkbnr/pppp1ppp/8/4p3/8/8/PPPPPPPP/RNBQKBNR
Active Color: w
Castling Rights: KQkq
En Passant Target: e6
Halfmove Clock: 0
Fullmove Number: 2
Input Tensor: tensor([[2, 3, 4, 6, 5, 4, 3, 2],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7]])


In [25]:
def UpdateInputWithSense(input_tensor, my_color, sensing_result):
    my_color = 'white'
    fen_pattern = re.compile(r"^([rnbqkpRNBQKP1-8/]+) ([wb]) ([KQkq-]+) ([a-h1-8-]+) (\d+) (\d+)$")
    if my_color == 'white':
        piece2encoding = {
            'p' : -1,
            'r' : -2,
            'n' : -3,
            'b' : -4,
            'q' : -5,
            'k' : -6,
            'e' : 0,
            'P' : 1,
            'R' : 2,
            'N' : 3,
            'B' : 4,
            'Q' : 5,
            'K' : 6,
        }
    elif my_color == 'black':
        piece2encoding = {
            'P' : -1,
            'R' : -2,
            'N' : -3,
            'B' : -4,
            'Q' : -5,
            'K' : -6,
            'e' : 0,
            'p' : 1,
            'r' : 2,
            'n' : 3,
            'b' : 4,
            'q' : 5,
            'k' : 6,
        }
    for (idx, piece) in sensing_result:
        rank, file = divmod(idx, 8)

        if piece != None:
            input_tensor[rank][file] = piece2encoding[piece.symbol()]
        else:
            input_tensor[rank][file] = piece2encoding['e']

    return input_tensor

sensing_result = game_history.sense_result(arbitrary_move)
print(sensing_result)

print(UpdateInputWithSense(input_tensor, my_color = 'white', sensing_result = sensing_result))

[(24, None), (25, None), (26, None), (16, None), (17, None), (18, None), (8, Piece.from_symbol('P')), (9, Piece.from_symbol('P')), (10, Piece.from_symbol('P'))]
tensor([[2, 3, 4, 6, 5, 4, 3, 2],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 7, 7, 7, 7, 7],
        [0, 0, 0, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7]])


In [28]:
def replace_numbers_with_e(strings):
    def replacer(match):
        return 'e' * int(match.group())  # Convert number to integer and repeat 'e' that many times

    return [re.sub(r'\d+', replacer, s) for s in strings]

# Write function to convert FEN string to Output Tensor
def FEN2OutputTensor(fen_string, my_color):
    #my_color = 'white'
    fen_pattern = re.compile(r"^([rnbqkpRNBQKP1-8/]+) ([wb]) ([KQkq-]+) ([a-h1-8-]+) (\d+) (\d+)$")
    if my_color == 'white':
        opp_piece2channel_idx = {
            'e' : 0,
            'P' : 1,
            'R' : 2,
            'N' : 3,
            'B' : 4,
            'Q' : 5,
            'K' : 6,
            'p' : -1,
            'r' : -2,
            'n' : -3,
            'b' : -4,
            'q' : -5,
            'k' : -6
        }
    elif my_color == 'black':
        opp_piece2channel_idx = {
            'e' : 0,   # Empty space
            'p' : 1,   # My pawn
            'r' : 2,   # My rook
            'n' : 3,   # My knight
            'b' : 4,   # My bishop
            'q' : 5,   # My queen
            'k' : 6,   # My king
            'P' : 7,   # Opp pawn
            'R' : 8,   # Opp rook
            'N' : 9,   # Opp knight
            'B' : 10,  # Opp bishop
            'Q' : 11,  # Opp queen
            'K' : 12   # Opp king
        }



    
    match = fen_pattern.match(fen_string)
    
    if match:
        piece_placement, active_color, castling, en_passant, halfmove, fullmove = match.groups()
        print("Piece Placement:", piece_placement)
        print("Active Color:", active_color)
        print("Castling Rights:", castling)
        print("En Passant Target:", en_passant)
        print("Halfmove Clock:", halfmove)
        print("Fullmove Number:", fullmove)

        pieces_by_row = piece_placement.split('/')

        pieces_by_row = replace_numbers_with_e(pieces_by_row)
        print(f'modified fen string: {pieces_by_row}')

        output_tensor = torch.zeros((8, 8, 6 + 6 + 1))

        for (rank, fen_row) in zip(reversed(range(input_tensor.shape[0])), pieces_by_row):
            for (file, piece) in zip(reversed(range(input_tensor.shape[1])), fen_row):
                output_tensor[rank][file][opp_piece2channel_idx[piece]] = 1
        return output_tensor
        
    else:
        print("Invalid FEN string")


out = FEN2OutputTensor(game_history.truth_fen_before_move(arbitrary_move), my_color = 'white')
print(out[:, :, 12])

Piece Placement: rnbqkbnr/pppp1ppp/8/4p3/8/8/PPPPPPPP/RNBQKBNR
Active Color: w
Castling Rights: KQkq
En Passant Target: e6
Halfmove Clock: 0
Fullmove Number: 2
modified fen string: ['rnbqkbnr', 'ppppeppp', 'eeeeeeee', 'eeeepeee', 'eeeeeeee', 'eeeeeeee', 'PPPPPPPP', 'RNBQKBNR']
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


In [8]:
def hide_opponent_pieces(boardstate, turn_player):
    opp_pieces = None
    
    if turn_player == 'White':
        opp_pieces = ['p', 'r', 'n', 'b', 'q', 'k']
        

In [9]:
def add_to_history(input_history, new_input, max_seq_len = 100):
    input_history.append(new_input)

    if len(input_history) == 101:
        input_history = input_history[:-1]

    return

In [17]:
game_history = GameHistory.from_file("./RandomBot-TroutBot-black-2025_03_18-06_50_14.json")
max_seq_len = 100
input_history = []
input_seqs = []
outputs = []


nturns = game_history.num_turns()
turns = list(game_history.turns())

for turn in turns:
    if i % 2 == 0:
        turn_player = "white"
    else:
        turn_player = "black"
    
    postsense_input = None

    # Ture boardstate pre-sense
    true_boardstate = game_history.truth_fen_before_move(turn)
    output = FEN2OutputTensor(true_boardstate, my_color = turn_player)

    # Known boardstate pre-sense
    presense_input = FEN2InputTensor(true_boardstate, my_color = turn_player)

    
    
    # Sensing action?
    if game_history.has_sense(turn):
        sensing_result = game_history.sense_result(turn)
        postsense_input = UpdateInputWithSense(presense_input, my_color = turn_player, sensing_result)

    add_to_history(input_history, presense_input, max_seq_len = max_seq_len)
    
    input_seqs.append(torch.stack(input_history, dim = 0))
    outputs.append(output)

    if postsense_input is not None:
        add_to_history(input_history, postsense_input, max_seq_len = max_seq_len)
        input_seqs.append(torch.stack(input_history, dim = 0))
        outputs.append(output)

    # TODO: Store input_seqs and outputs into a Pytorch dataset.
    dataset = RBCDataset(input_seqs, outputs)
    torch.save(dataset, "dummy_dataset.pth")

    

Piece Placement: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR
Active Color: w
Castling Rights: KQkq
En Passant Target: -
Halfmove Clock: 0
Fullmove Number: 1
modified fen string: ['rnbqkbnr', 'pppppppp', 'eeeeeeee', 'eeeeeeee', 'eeeeeeee', 'eeeeeeee', 'PPPPPPPP', 'RNBQKBNR']
Input string: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
Piece Placement: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR
Active Color: w
Castling Rights: KQkq
En Passant Target: -
Halfmove Clock: 0
Fullmove Number: 1
Input Tensor: tensor([[2, 3, 4, 6, 5, 4, 3, 2],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7]])
Piece Placement: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR
Active Color: b
Castling Rights: KQkq
En Passant Target: -
Halfmove Clock: 1
Fullmove Number: 1
modified fen string: ['rnbqkbnr', 'pppppppp', 'eeeeee

In [35]:
max_seq_len = 100


def Game2Dataset(fname, max_seq_len = 100):
    game_history = GameHistory.from_file(fname)
    fname = fname.split('.')[0]
    input_history = []
    input_seqs = []
    outputs = []
    
    
    nturns = game_history.num_turns()
    turns = list(game_history.turns())
    
    for turn in turns:
        if i % 2 == 0:
            turn_player = "white"
        else:
            turn_player = "black"
        
        postsense_input = None
    
        # Ture boardstate pre-sense
        true_boardstate = game_history.truth_fen_before_move(turn)
        output = FEN2OutputTensor(true_boardstate, my_color = turn_player)
    
        # Known boardstate pre-sense
        presense_input = FEN2InputTensor(true_boardstate, my_color = turn_player)
    
        
        
        # Sensing action?
        if game_history.has_sense(turn):
            sensing_result = game_history.sense_result(turn)
            postsense_input = UpdateInputWithSense(presense_input, my_color = turn_player, sensing_result = sensing_result)
    
        add_to_history(input_history, presense_input, max_seq_len = max_seq_len)
        
        input_seqs.append(torch.stack(input_history, dim = 0))
        outputs.append(output)
    
        if postsense_input is not None:
            add_to_history(input_history, postsense_input, max_seq_len = max_seq_len)
            input_seqs.append(torch.stack(input_history, dim = 0))
            outputs.append(output)
    
        dataset = RBCDataset(input_seqs, outputs)
        torch.save(dataset, f"{fname}.pth")

Game2Dataset("RandomBot-TroutBot-black-2025_03_18-06_50_14.json", max_seq_len)

    

Piece Placement: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR
Active Color: w
Castling Rights: KQkq
En Passant Target: -
Halfmove Clock: 0
Fullmove Number: 1
modified fen string: ['rnbqkbnr', 'pppppppp', 'eeeeeeee', 'eeeeeeee', 'eeeeeeee', 'eeeeeeee', 'PPPPPPPP', 'RNBQKBNR']
Input string: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
Piece Placement: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR
Active Color: w
Castling Rights: KQkq
En Passant Target: -
Halfmove Clock: 0
Fullmove Number: 1
Input Tensor: tensor([[2, 3, 4, 6, 5, 4, 3, 2],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7, 7]])
Piece Placement: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR
Active Color: b
Castling Rights: KQkq
En Passant Target: -
Halfmove Clock: 1
Fullmove Number: 1
modified fen string: ['rnbqkbnr', 'pppppppp', 'eeeeee

In [36]:
# Testing saved dataset

def collate_fn(batch):
    sequences, targets = zip(*batch)
    sequences = pad_sequence(sequences, batch_first=True, padding_value=0.0)
    targets = torch.stack(targets)
    return sequences, targets


dataset = torch.load("RandomBot-TroutBot-black-2025_03_18-06_50_14.pth")
loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

for x_batch, y_batch in loader:
    print("Batch input shape:", x_batch.shape)
    print("Batch target shape:", y_batch.shape)
    break 

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL preprocessing.RBCDataset.RBCDataset was not an allowed global by default. Please use `torch.serialization.add_safe_globals([RBCDataset])` or the `torch.serialization.safe_globals([RBCDataset])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by def